In [30]:
import time
from time import sleep
import traceback
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import  Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import os
import sys


def login(navegador):
    navegador.get("https://sei.rj.gov.br/sei/controlador.php?acao=procedimento_trabalhar&acao_origem=rel_bloco_protocolo_listar&acao_retorno=rel_bloco_protocolo_listar&id_procedimento=81745334&id_bloco=847496&infra_sistema=100000100&infra_unidade_atual=110000196&infra_hash=4fa73d8fa9f90607eadfa66760acc9ab5aee3147ef7e204d92847ab0f335cde5")

    usuario = navegador.find_element(By.XPATH, value='//*[@id="txtUsuario"]')
    usuario.send_keys(os.environ['login_sefaz'])

    senha = navegador.find_element(By.XPATH, value='//*[@id="pwdSenha"]')
    senha.send_keys(os.environ['senha_sefaz'])

    exercicio = Select(navegador.find_element(By.XPATH, value='//*[@id="selOrgao"]'))
    exercicio.select_by_visible_text('SEFAZ')

    btnLogin = navegador.find_element(By.XPATH, value='//*[@id="Acessar"]')
    btnLogin.click()

    
    WebDriverWait(navegador,20).until(EC.presence_of_element_located((By.ID, "divInfraBarraSistemaPadrao" )))

    

    
    navegador.maximize_window()

    
    
    
    navegador.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)


navegador = webdriver.Firefox()


In [31]:
login(navegador)


In [8]:
import re
encontrarProcessos(navegador,"863135")
#encontrarFormaDePagamento(navegador)
#encontrarValidade(navegador)

Beneficiário: TJRJ / CNPJ: 28.538.734/0001-48
Forma de Pagamento: GRERJ (76504393)


''

In [1]:
import pandas as pd
from openpyxl import load_workbook
import traceback
bloco = "4"
wb = load_workbook(r"C:\Users\jcampbell1\OneDrive - SEFAZ-RJ\CONTROLE GERENCIAL - PAGAMENTOS\Planilha Gerencial - Marinette.xlsx")

if bloco not in wb.sheetnames:
    wb.create_sheet(bloco,0)
    wb.save(r"C:\Users\jcampbell1\OneDrive - SEFAZ-RJ\CONTROLE GERENCIAL - PAGAMENTOS\Planilha Gerencial - Marinette.xlsx")

df = pd.read_excel(r"C:\Users\jcampbell1\OneDrive - SEFAZ-RJ\CONTROLE GERENCIAL - PAGAMENTOS\Planilha Gerencial - Marinette.xlsx", sheet_name=bloco)

try:
    print(df["PROCESSO"].values)

except:
    df = pd.DataFrame(columns=["PROCESSO", "FORMA DE PAGAMENTO", "VALIDADE","ACOMPANHAMENTO ESPECIAL"], index=None)

writer = pd.ExcelWriter(r"C:\Users\jcampbell1\OneDrive - SEFAZ-RJ\CONTROLE GERENCIAL - PAGAMENTOS\Planilha Gerencial - Marinette.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace')

df.to_excel(writer, sheet_name=bloco, index=False)

writer.close()


: 

In [3]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import CellIsRule

marinette = r"C:\Users\jcampbell1\OneDrive - SEFAZ-RJ\CONTROLE GERENCIAL - PAGAMENTOS\Planilha Gerencial - Marinette.xlsx"
bloco = "869143"
df = pd.read_excel(marinette, sheet_name=bloco)


writer = pd.ExcelWriter(marinette, engine='openpyxl', mode='a', if_sheet_exists='replace')
df.to_excel(writer, sheet_name=bloco, index=False)
writer.close()

planilha = load_workbook(marinette)
tabela = planilha[bloco]
for linha in range(2,tabela.max_row + 1):
    celula = tabela[f"D{linha}"]
    celula.value = f'=C{linha}-TODAY()'



prazo = f'D2:D{tabela.max_row}'


red_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
yellow_fill = PatternFill(start_color="FFFFE0", end_color="FFFFE0", fill_type="solid")

    
red_rule = CellIsRule(operator='lessThan', formula=['0'], stopIfTrue=True, fill=red_fill)
    
# Rule for cells with positive values (green)
green_rule = CellIsRule(operator='greaterThan', formula=['15'], stopIfTrue=True, fill=green_fill)
yellow_rule = CellIsRule(operator='lessThanOrEqual', formula=['15'], stopIfTrue=True, fill=yellow_fill)

# Add rules to the worksheet
tabela.conditional_formatting.add(prazo, red_rule)
tabela.conditional_formatting.add(prazo, green_rule)


planilha.save(marinette)
planilha.close()

: 

In [10]:
navegador.switch_to.default_content()
WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))
docs = navegador.find_elements(By.XPATH, "//div[@id = 'divArvore']//div//a[@class = 'infraArvoreNo']")
quantDocs = len(docs)
for doc in range(quantDocs):
    docTexto = docs[doc].text
    if "Guia" in docTexto or "GRERJ" in docTexto:
        docs[doc].click()
        time.sleep(2)
        
        navegador.switch_to.default_content()            
        WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrVisualizacao")))
        WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvoreHtml")))
        spans = navegador.find_elements(By.XPATH, '//span')
        guia = ""
        for span in spans:

            
            if "BANCO DO BRASIL" in span.get_attribute("innerHTML").upper():
                guia = "Guia BB" 
                break
            if "GRERJ" in span.get_attribute("innerHTML").upper():
                guia = "Guia GRERJ"
                break
            
        if guia == "Guia BB" or guia == "Guia GRERJ" :
            for span in spans:
                #Regex pra achar as datas
                regex = re.match("^\d{2}\/\d{2}\/\d{4}$",span.get_attribute('innerHTML'))
                if regex:
                    validade = regex.group()
                    print(validade)
                    #return validade,guia
        
        navegador.switch_to.default_content()            
        WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))
        docs = navegador.find_elements(By.XPATH, "//div[@id = 'divArvore']//div//a[@class = 'infraArvoreNo']")   


26/06/2024
04/07/2024


In [18]:
spans = navegador.find_elements(By.XPATH, '//span')
##PRIMEIRO DESCOBRIR SE É BRASIL
## LEMBRAR QUAL ATRIBUTO É DO SPAN
for span in spans:
    if "BANCO DO BRASIL" in span.get_attribute("innerHTML").upper():
        guia = "Guia BB" 
        break
    
if guia == "Guia BB":
    for span in spans:
        #Regex pra achar as datas
        regex = re.match("^\d{2}\/\d{2}\/\d{4}$",span.get_attribute('innerHTML'))
        if regex:
            validade = regex.group()
            break
validade

'12/06/2024'

In [6]:
def encontrarValidade(navegador):
    navegador.switch_to.default_content()
    WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))
    docs = navegador.find_elements(By.XPATH, "//div[@id = 'divArvore']//div//a[@class = 'infraArvoreNo']")
    for doc in docs:
        docTexto = doc.text
        if "Despacho de Encaminhamento de Processo" in docTexto:
            doc.click()
            time.sleep(2)
            
            navegador.switch_to.default_content()            
            WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrVisualizacao")))
            WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvoreHtml")))
            try:
                tabelaGRE = navegador.find_element(By.XPATH, "//table[@id = 'tabelaConsultaGRE']")
                validade = tabelaGRE.find_elements(By.XPATH, ".//td")[7]
                print(validade.text)
                return validade
                
            except:
                navegador.switch_to.default_content()
                WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))

                pass


In [4]:

def encontrarProcessos(navegador,blocoSolicitado):
    navegador.find_element(By.XPATH, "//span[text() = 'Blocos']").click()
    WebDriverWait(navegador,20).until(EC.element_to_be_clickable((By.XPATH, "//span[text() = 'Internos']"))).click()
    blocos = navegador.find_elements(By.XPATH, "//tbody//tr")[1:-1]

    for bloco in blocos:    
        nBloco = bloco.find_elements(By.XPATH,".//td")[1]
        if nBloco.text == blocoSolicitado:
            nBloco.find_element(By.XPATH, './/a').click()
            break
    processos = navegador.find_elements(By.XPATH, "//tbody//tr")[1:]

    for i in range(1,len(processos)):
        processo = navegador.find_elements(By.XPATH, "//tbody//tr")[i]
        WebDriverWait(processo,20).until(EC.element_to_be_clickable(((By.XPATH, './/td[3]//a'))))
        if processo.find_element(By.XPATH, './/td[3]//a').text == "SEI-040009/000757/2024":

            processo.find_element(By.XPATH, './/td[3]//a').click()
            break

In [7]:
def encontrarFormaDePagamento(navegador):

    navegador.switch_to.window(navegador.window_handles[1])
    WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))
    docs = navegador.find_elements(By.XPATH, "//div[@id = 'divArvore']//div//a[@class = 'infraArvoreNo']")

    for doc in docs:
        docTexto = doc.text
        if "Despacho sobre Autorização de Despesa" in docTexto:
            
            
            doc.click()
            time.sleep(2)
            
            navegador.switch_to.default_content()            
            WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrVisualizacao")))
            WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvoreHtml")))
            try:
                beneficiario = navegador.find_element(By.XPATH, "//p//strong[contains(text(), 'Beneficiário')]" )
                print(beneficiario.text)
                if  "CPF" in beneficiario.text:
                    formaPagamento = "Depósito"
                    
                    
                
                    return formaPagamento


                

                


                formaPagamentoDespacho = navegador.find_element(By.XPATH, "//p//strong[contains(text(), 'Forma de Pagamento')]" )
                print(formaPagamentoDespacho.text)
                
                formaPagamento = ""

                if "DEPÓSITO JUDICIAL" in formaPagamentoDespacho.text:
                    formaPagamento = "Guia"
                if "GUIA" in formaPagamentoDespacho.text:
                    formaPagamento = "Guia"



                navegador.switch_to.default_content()

                return formaPagamento
            


            except:
                traceback.print_exc()
